# tt̄ spin observables: Data/MC stacks + P,C,D extraction
This notebook stacks Data vs MC histograms and extracts spin observables from cosine histograms.

**Features**
- Uses PyROOT when available; otherwise falls back to uproot.
- Scales each MC by `xsec*lumi/n_events` from `plots/count`.
- Plots stacks with mplhep; highlights TTTo2L2Nu in red.
- Extracts `<cosθ+> → P_a`, `<cosθ+ cosθ-> → C_aa`, and `D=(Ckk+Crr+Cnn)/3` per channel.


## 0) Setup & imports

In [1]:
import os, math, numpy as np, sys, csv
from pathlib import Path
import matplotlib.pyplot as plt
import mplhep as hep
USE_ROOT = True
try:
    import ROOT
    ROOT.gErrorIgnoreLevel = ROOT.kError
    ROOT.TH1.SetDefaultSumw2(True)
except Exception as e:
    USE_ROOT = False
    import uproot
out_dir = 'compare_plots_mpl_stack'
os.makedirs(out_dir, exist_ok=True)
print('Backend:', 'PyROOT' if USE_ROOT else 'uproot')


Backend: PyROOT


## 1) File lists, xsecs, lumi

In [2]:
data_files = [
    'DoubleMuon_Data_2018.root',
    'MuonEG_Data_2018.root',
    'EGamma_Data_2018.root',
    'SingleMuon_Data_2018.root',
]
mc_files = [
    'DYJetsToLL_M-10to50_MC_2018.root',
    'DYJetsToLL_M-50_MC_2018.root',
    'ST_s-channel_MC_2018.root',
    'ST_t-channel_antitop_MC_2018.root',
    'ST_t-channel_top_MC_2018.root',
    'ST_tW_antitop_MC_2018.root',
    'ST_tW_top_MC_2018.root',
    'TTTo2L2Nu_MC_2018.root',
    'TTToHadronic_MC_2018.root',
    'TTToSemiLeptonic_MC_2018.root',
    'WJetsToLNu_MC_2018.root',
    'WW_MC_2018.root',
    'WZ_MC_2018.root',
    'ZZ_MC_2018.root',
]
cross_sections_pb = {
    'DYJetsToLL_M-10to50_MC_2018.root': 18610.0,
    'DYJetsToLL_M-50_MC_2018.root':    6077.22,
    'ST_s-channel_MC_2018.root':       3.36,
    'ST_t-channel_antitop_MC_2018.root': 80.95,
    'ST_t-channel_top_MC_2018.root':   136.02,
    'ST_tW_antitop_MC_2018.root':      34.91,
    'ST_tW_top_MC_2018.root':          34.97,
    'TTTo2L2Nu_MC_2018.root':          87.4,
    'TTToHadronic_MC_2018.root':       380.0,
    'TTToSemiLeptonic_MC_2018.root':   364.0,
    'WJetsToLNu_MC_2018.root':         61526.0,
    'WW_MC_2018.root':                 118.7,
    'WZ_MC_2018.root':                 47.13,
    'ZZ_MC_2018.root':                 16.523,
}
luminosity_pb = 59.740 * 1000.0
alpha_l = 1.0


## 1.5) Histogram discovery

In [3]:
def get_hist_names(filename, directory='plots'):
    if USE_ROOT:
        f = ROOT.TFile.Open(filename)
        if not f or f.IsZombie():
            return []
        if not f.GetDirectory(directory):
            f.Close(); return []
        f.cd(directory)
        names = [k.GetName() for k in ROOT.gDirectory.GetListOfKeys()]
        f.Close()
        return names
    else:
        with uproot.open(filename) as f:
            if directory not in f:
                return []
            return [k.split(';')[0] for k in f[directory].keys()]

hist_names = get_hist_names(mc_files[0])
prefixes = ('mumu','ee','emu','combine','numberof','ll','all')
hist_names = [h for h in hist_names if any(h.startswith(p) for p in prefixes)]
print('Found', len(hist_names),'histograms')


Found 257 histograms


## 2) Helpers to read/scale and convert to numpy

In [4]:
def read_hist(file_name, hist_path):
    if USE_ROOT:
        f = ROOT.TFile.Open(file_name)
        if not f or f.IsZombie():
            return None
        h = f.Get(hist_path)
        if h:
            h = h.Clone(f'tmp_{os.path.basename(file_name)}_{h.GetName()}')
            h.SetDirectory(0)
        f.Close(); return h
    else:
        with uproot.open(file_name) as f:
            if hist_path not in f:
                return (None, None)
            h = f[hist_path]
            try:
                edges = h.axis().edges(); counts = h.values()
            except Exception:
                counts, edges = h.to_numpy()
            return (np.asarray(edges), np.asarray(counts))

def read_count(file_name, count_path='plots/count'):
    if USE_ROOT:
        f = ROOT.TFile.Open(file_name)
        if not f or f.IsZombie():
            return 0.0
        cnt = f.Get(count_path)
        n = cnt.GetSumOfWeights() if cnt else 0.0
        f.Close(); return float(n)
    else:
        with uproot.open(file_name) as f:
            if count_path not in f: return 0.0
            cnt = f[count_path]
            try:
                return float(cnt.values().sum())
            except Exception:
                try:
                    return float(cnt.member('fTsumw'))
                except Exception:
                    return 0.0

def hist_to_numpy_root(h):
    nb = h.GetNbinsX()
    edges = np.array([h.GetBinLowEdge(i) for i in range(1, nb+1)] + [h.GetBinLowEdge(nb)+h.GetBinWidth(nb)])
    counts = np.array([h.GetBinContent(i) for i in range(1, nb+1)])
    return edges, counts

def stack_mc_numpy(hname):
    hist_path = f'plots/{hname}'
    arrays, labels = [], []
    edges_common = None
    mc_sum_hist = None
    for fn in mc_files:
        xsec = cross_sections_pb.get(fn, 1.0)
        if USE_ROOT:
            h = read_hist(fn, hist_path)
            if not h: continue
            n_evt = read_count(fn)
            if n_evt <= 0: continue
            scale = (xsec * luminosity_pb) / n_evt
            h.Sumw2(); h.Scale(scale)
            edges, counts = hist_to_numpy_root(h)
            if edges_common is None: edges_common = edges
            arrays.append(counts); labels.append(os.path.splitext(fn)[0])
            if mc_sum_hist is None:
                mc_sum_hist = h.Clone('mc_sum'); mc_sum_hist.SetDirectory(0)
            else:
                mc_sum_hist.Add(h)
        else:
            edges, counts = read_hist(fn, hist_path)
            if edges is None: continue
            n_evt = read_count(fn)
            if n_evt <= 0: continue
            scale = (xsec * luminosity_pb) / n_evt
            counts = counts * scale
            if edges_common is None: edges_common = edges
            arrays.append(counts); labels.append(os.path.splitext(fn)[0])
    if edges_common is None or len(arrays)==0:
        return None, [], [], None, None
    arrays = [np.asarray(a) for a in arrays]
    mc_sum_counts = np.sum(arrays, axis=0) if len(arrays)>0 else None
    return edges_common, arrays, labels, mc_sum_counts, mc_sum_hist


## 3) Combine data helper

In [5]:
def combine_data(hname):
    hist_path = f'plots/{hname}'
    if USE_ROOT:
        data_hist = None
        for fn in data_files:
            h = read_hist(fn, hist_path)
            if not h: continue
            if data_hist is None:
                data_hist = h.Clone('data_sum'); data_hist.SetDirectory(0)
            else:
                data_hist.Add(h)
        if data_hist is None: return (None, None)
        return hist_to_numpy_root(data_hist)
    else:
        edges, counts = None, None
        for fn in data_files:
            e, c = read_hist(fn, hist_path)
            if e is None: continue
            if edges is None:
                edges, counts = e, c.copy()
            else:
                if not np.allclose(edges, e):
                    raise RuntimeError('Inconsistent data binning')
                counts += c
        return edges, counts


## 4) cosθ helpers

In [6]:
def is_cos_hist(hname):
    keys = [
        'cosThetaPlus_k','cosThetaMinus_k','cosThetaPlus_k_times_Minus_k',
        'cosThetaPlus_r','cosThetaMinus_r','cosThetaPlus_r_times_Minus_r',
        'cosThetaPlus_n','cosThetaMinus_n','cosThetaPlus_n_times_Minus_n',
    ]
    return any(k in hname for k in keys)

def axis_from_name(hname):
    if '_k' in hname: return 'k'
    if '_r' in hname: return 'r'
    if '_n' in hname: return 'n'
    return None

def channel_from_name(hname):
    for ch in ['ee','mumu','emu','all']:
        if hname.startswith(ch+'_'): return ch
    return 'all'

def mean_from_counts(edges, counts):
    centers = 0.5*(edges[:-1]+edges[1:])
    tot = counts.sum()
    if tot <= 0: return float('nan')
    return float(np.dot(centers, counts)/tot)


## 5) Main loop: stacks + observables

In [7]:
plt.style.use(hep.style.CMS)
results = []
for hname in hist_names:
    # 1D guard (best-effort for uproot case)
    # Data
    edges, dat_counts = combine_data(hname)
    if edges is None:
        print('[skip] No data for', hname); continue
    dat_err = np.sqrt(np.clip(dat_counts,0,None))
    centers = 0.5*(edges[:-1]+edges[1:])

    # MC
    edges_mc, mc_arrays, mc_labels, mc_sum_counts, mc_sum_hist = stack_mc_numpy(hname)
    if edges_mc is None:
        print('[skip] No MC for', hname); continue

    # Colors
    cmap = plt.get_cmap('tab20')
    others = [lbl for lbl in mc_labels if lbl != 'TTTo2L2Nu_MC_2018']
    color_map = {lbl: cmap(i/max(1,len(others)-1)) for i,lbl in enumerate(others)}
    color_map['TTTo2L2Nu_MC_2018'] = 'red'
    mc_colors = [color_map[lbl] for lbl in mc_labels]

    # Plot
    fig, (ax, axr) = plt.subplots(nrows=2, sharex=True,
        gridspec_kw={'height_ratios':[3,1],'hspace':0.05}, figsize=(10,8), dpi=150)
    hep.histplot(mc_arrays, bins=edges, stack=True, histtype='fill', label=mc_labels, color=mc_colors, ax=ax)
    ax.errorbar(centers, dat_counts, yerr=dat_err, fmt='o', color='black', label='Data')
    ax.set_ylabel('Events / bin')
    hep.cms.label('Private Work', data=True, lumi=59.8, ax=ax)
    ax.legend(loc='upper right', prop={'size':8}, handletextpad=0.2, labelspacing=0.2, columnspacing=0.5)

    mc_sum = mc_sum_counts
    mask = mc_sum > 0
    ratio = np.divide(dat_counts, mc_sum, out=np.zeros_like(dat_counts), where=mask)
    ratio_err = np.divide(dat_err, mc_sum, out=np.zeros_like(dat_err), where=mask)
    axr.errorbar(centers[mask], ratio[mask], yerr=ratio_err[mask], fmt='o', color='black')
    axr.axhline(1, color='gray', linestyle='--', linewidth=1)
    axr.set_ylabel('Data/MC'); axr.set_xlabel(hname); axr.set_ylim(0.5,1.5)

    fig.savefig(f"{out_dir}/stack_{hname}.png"); plt.close(fig)

    # Observables
    if is_cos_hist(hname):
        ch = channel_from_name(hname)
        axis = axis_from_name(hname)
        mean_data = mean_from_counts(edges, dat_counts)
        if USE_ROOT and mc_sum_hist is not None:
            mean_mc = float(mc_sum_hist.GetMean())
        else:
            mean_mc = mean_from_counts(edges, mc_sum_counts)
        if 'times_Minus_' in hname:
            C_data = (9.0/(alpha_l**2))*mean_data
            C_mc   = (9.0/(alpha_l**2))*mean_mc
            results.append([hname,ch,axis,'Caa',mean_data,mean_mc,C_data,C_mc])
        elif 'cosThetaPlus_' in hname:
            P_data = (3.0/alpha_l)*mean_data
            P_mc   = (3.0/alpha_l)*mean_mc
            results.append([hname,ch,axis,'Pa',mean_data,mean_mc,P_data,P_mc])

# Build D per channel
C_map_data = {ch:{'k':None,'r':None,'n':None} for ch in ['ee','mumu','emu','all']}
C_map_mc   = {ch:{'k':None,'r':None,'n':None} for ch in ['ee','mumu','emu','all']}
for hname,ch,axis,kind,mean_d,mean_m,val_d,val_m in results:
    if kind=='Caa':
        C_map_data[ch][axis]=val_d; C_map_mc[ch][axis]=val_m
for ch in C_map_data:
    cds=C_map_data[ch]; cms=C_map_mc[ch]
    if all(v is not None for v in cds.values()):
        D_data=(cds['k']+cds['r']+cds['n'])/3.0
        results.append([f'{ch}_D',ch,'sum','D',float('nan'),float('nan'),D_data,float('nan')])
    if all(v is not None for v in cms.values()):
        D_mc=(cms['k']+cms['r']+cms['n'])/3.0
        results.append([f'{ch}_D_mc',ch,'sum','D_MC',float('nan'),float('nan'),float('nan'),D_mc])

# Save CSV
import pandas as pd
df = pd.DataFrame(results, columns=['hname','channel','axis','kind','mean_data','mean_mc','value_data','value_mc'])
csv_path = f"{out_dir}/spin_observables.csv"
df.to_csv(csv_path, index=False)
print('Saved:', csv_path)


Saved: compare_plots_mpl_stack/spin_observables.csv


## 6) Preview CSV (if available)

In [8]:
import pandas as pd, os
csv_path = f"{out_dir}/spin_observables.csv"
if os.path.exists(csv_path):
    display(pd.read_csv(csv_path).head(20))
else:
    print('Run the previous cell after pointing to real files to produce the CSV.')


,hname,channel,axis,kind,mean_data,mean_mc,value_data,value_mc
0,ee_cosThetaPlus_k,ee,k,Pa,-0.038832,-0.156229,-0.116496,-0.468688
1,ee_cosThetaPlus_k_times_Minus_k,ee,k,Caa,0.052737,0.012439,0.474631,0.111950
2,ee_cosThetaPlus_r,ee,r,Pa,0.006837,-0.026162,0.020511,-0.078486
3,ee_cosThetaPlus_r_times_Minus_r,ee,r,Caa,0.027288,0.036251,0.245592,0.326259
4,ee_cosThetaPlus_n,ee,n,Pa,0.000013,-0.091750,0.000038,-0.275250
5,ee_cosThetaPlus_n_times_Minus_n,ee,n,Caa,0.035387,0.071179,0.318485,0.640610
6,mumu_cosThetaPlus_k,mumu,k,Pa,-0.057023,-0.242744,-0.171068,-0.728231
7,mumu_cosThetaPlus_k_times_Minus_k,mumu,k,Caa,0.052643,0.103421,0.473783,0.930789
8,mumu_cosThetaPlus_r,mumu,r,Pa,0.004253,-0.007646,0.012759,-0.022938
9,mumu_cosThetaPlus_r_times_Minus_r,mumu,r,Caa,0.032841,0.062963,0.295567,0.566669
